In [4]:
import time
import numpy as np
import pinocchio as pin
import os
import meshcat
import matplotlib.pyplot as plt
import scipy

np.set_printoptions(suppress=True, precision=4) 
package_dirs = './urdf/' 
urdf_file = 'tara.urdf'  

urdf = package_dirs + urdf_file
robot = pinocchio.RobotWrapper.BuildFromURDF(urdf, package_dirs)
END_EFF_FRAME_ID = robot.model.getFrameId('end_effector') 
viz = pinocchio.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer")
    print(err)
    sys.exit(0)
viz.loadViewerModel()
q = pinocchio.neutral(robot.model)
viz.display(q) 

def compute_error(desired_pos, current_pos):
    return desired_pos - current_pos
    
def inverse_kinematics(robot, q_init, desired_pos, max_iters=100, tol=1e-4, alpha=0.1):
    # Start with the initial guess for joint angles
    q = q_init.copy()  
    
    for i in range(max_iters):
        pin.forwardKinematics(robot.model, robot.data, q)
        pin.updateFramePlacements(robot.model, robot.data)
        current_pos = robot.data.oMf[END_EFF_FRAME_ID].translation
        error = compute_error(desired_pos, current_pos)
        if np.linalg.norm(error) < tol:
            print(f"Converged in {i+1} iterations.")
            return q
        
        # Compute the Jacobian of the end-effector
        J = pin.computeFrameJacobian(robot.model, robot.data, q, END_EFF_FRAME_ID, pin.LOCAL_WORLD_ALIGNED)[:3, :]
        
        q += alpha * np.linalg.pinv(J).dot(error)
    
    print(f"Reached max iterations. Final error: {np.linalg.norm(error)}")
    return q

q_init = pin.neutral(robot.model)
desired_position = np.array([1, 1, 1])
q_solution = inverse_kinematics(robot, q_init, desired_position)
viz.display(q_solution)
print("Final joint configuration:", q_solution)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7016/static/
Converged in 92 iterations.
Final joint configuration: [0.7854 0.4561 1.4537 0.    ]
